In [1]:
import os
import glob
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

import tensorflow as tf
import yolo.config as cfg
from utils.help import say

Using TensorFlow backend.


In [2]:
sess = tf.Session()
K.set_session(sess)

K.set_image_dim_ordering('th')
weights_path = 'yolo-tiny-origin.h5'
is_freeze = True
verbalise = True

In [3]:
def make_yolotiny_network(is_freeze=True):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(3,448,448),padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(64,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(128,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(256,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(512,(3,3), padding='same', 
                            activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Conv2D(1024,(3,3), padding='same', activation=LeakyReLU(alpha=0.1), trainable=not is_freeze))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))
    
    return model

In [4]:
def make_bottleneck_network(model):
    model.add(Dense(256, input_shape=model.output_shape))
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1470))
    
    return model

In [5]:
model = make_yolotiny_network()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [6]:
from yolo.training_v1 import darkeras_loss, _TRAINER
from yolo.datacenter.data import shuffle

### 처음에는 랜덤 이니셜라이즈가 된걸로 보임

In [8]:
sess.run(tf.global_variables_initializer())

inp_x = model.input
net_out = model.output

conv1 = model.layers[0]
dense_last = model.layers[-1]
print("conv1 : ")
print(sess.run(conv1.weights[0])[0][0][0])
print("danse last : ")
print(sess.run(dense_last.weights[0])[0][0:10])

conv1 : 
[-0.10797721  0.17980772  0.10209945  0.08317301 -0.16234875  0.01490965
 -0.09045276 -0.16318287  0.11130679  0.1397835  -0.15183559 -0.10002695
 -0.0053302  -0.07572366  0.05486624  0.10980004]
danse last : 
[ 0.02892222 -0.01035621 -0.02166484  0.01330325 -0.02072718 -0.02225607
  0.01895375  0.0305136   0.01021824 -0.00729086]


### 강제로 global initializer 를 시키면?

In [9]:
inp_x = model.input
net_out = model.output


say("Building {} loss function".format(cfg.model_name), verbalise=verbalise)
loss_ph, loss_op = darkeras_loss(net_out)
say("Building {} train optimizer".format(cfg.model_name), verbalise=verbalise)
optimizer = _TRAINER[cfg.trainer](cfg.lr)
gradients = optimizer.compute_gradients(loss_op)
train_op = optimizer.apply_gradients(gradients)


conv1 = model.layers[0]
dense_last = model.layers[-1]
print("conv1 : ")
print(sess.run(conv1.weights[0])[0][0][0])
print("danse last : ")
print(sess.run(dense_last.weights[0])[0][0:10])

['Building yolo-tiny loss function']
Building yolo-tiny loss
['Building yolo-tiny train optimizer']
conv1 : 
[-0.10797721  0.17980772  0.10209945  0.08317301 -0.16234875  0.01490965
 -0.09045276 -0.16318287  0.11130679  0.1397835  -0.15183559 -0.10002695
 -0.0053302  -0.07572366  0.05486624  0.10980004]
danse last : 
[ 0.02892222 -0.01035621 -0.02166484  0.01330325 -0.02072718 -0.02225607
  0.01895375  0.0305136   0.01021824 -0.00729086]


In [9]:
# model.load_weights(weights_path)
# say("Setting weigths : {}",format(weights_path), verbalise=verbalise)

# conv1 = model.layers[0]
# dense_last = model.layers[-1]
# print("conv1 : ")
# print(sess.run(conv1.weights[0])[0][0][0])
# print("danse last : ")
# print(sess.run(dense_last.weights[0])[0][0:10])

['Setting weigths : {}', 'yolo-tiny-origin.h5']
conv1 : 
[ 0.35519508 -0.22792202 -0.59281367  0.01867884  0.06647962  0.351217
  0.05631017  0.4962424   0.51120818  0.13387974 -0.09508803 -0.21474499
 -0.70027256  0.57914382  0.18591353 -0.28030169]
danse last : 
[  3.06502054e-03  -1.85736606e-03  -5.30227517e-05  -6.94487384e-03
  -1.01462668e-02  -3.37091595e-04   5.96690690e-03   2.54362170e-03
  -6.02794159e-03   3.24628293e-03]


In [10]:
# pop_layer = model.pop() # dense_25
# say("{} layer poped".format(pop_layer), verbalise=verbalise)
# pop_layer = model.pop() # leakyrelu_34
# say("{} layer poped".format(pop_layer), verbalise=verbalise)
# pop_layer = model.pop() # dense_24
# say("{} layer poped".format(pop_layer), verbalise=verbalise)
# pop_layer = model.pop() # dense 23
# say("{} layer poped".format(pop_layer), verbalise=verbalise)
# model.layers

['None layer poped']
['None layer poped']
['None layer poped']
['None layer poped']


In [10]:
# model = make_bottleneck_network(model)
# model.layers

In [10]:
model.load_weights(weights_path)
say("Setting weigths : {}",format(weights_path), verbalise=verbalise)

pop_layer = model.pop() # dense_25
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # leakyrelu_34
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense_24
say("{} layer poped".format(pop_layer), verbalise=verbalise)
pop_layer = model.pop() # dense 23
say("{} layer poped".format(pop_layer), verbalise=verbalise)

model.summary()
print(model.output_shape)

['Setting weigths : {}', 'yolo-tiny-origin.h5']
['None layer poped']
['None layer poped']
['None layer poped']
['None layer poped']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
__________

In [11]:
model = make_bottleneck_network(model)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 56, 56)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 56, 56)       73856     
__________

In [12]:
conv1 = model.layers[0]
conv1.built

True

In [14]:
batches = shuffle()
for i, (x_batch, datum) in enumerate(batches):
    train_feed_dict = {
       loss_ph[key]:datum[key] for key in loss_ph 
    }
    train_feed_dict[inp_x] = x_batch
    # print("feed_dict.keys() : ", len(train_feed_dict.keys()), train_feed_dict.keys())
    fetches = [train_op, loss_op] 
    fetched = sess.run(fetches, feed_dict=train_feed_dict)
    
    loss_val = fetched[1]
    say("step {} - loss {}".format(i, loss_val), verbalise=True)
    
    conv1 = model.layers[0]
    dense_last = model.layers[-1]
    print("conv1 : ")
    print(sess.run(conv1.weights[0])[0][0][0])
#     print("danse last : ")
#     print(sess.run(dense_last.weights[0])[0][0:10])
    
    if i == 1:
        save_step_weigths_path = 'yolo-tiny-step{}.h5'.format(i)
        model.save_weights(save_step_weigths_path)
        print("Saved weigths : ", save_step_weigths_path)
    if i % 310 == 0:
        model.save_weights('yolo-tiny-epoch{}.h5'.format(i//310))
        say("Save weights : ", 'yolo-tiny-epoch{}.h5'.format(i//310), verbalise=verbalise)

Dataset of 9963 instance(s)
shuffle index :  [ 436 5659 6995 ...,  239 2041 8592]
['step 0 - loss 2.8475615978240967']
conv1 : 
[ 0.35505235 -0.22811179 -0.59261441  0.01887837  0.06628548  0.35139766
  0.05650574  0.49642369  0.51140738  0.13368623 -0.09527325 -0.21493055
 -0.70031995  0.57934397  0.18571344 -0.28040144]
['Save weights : ', 'yolo-tiny-epoch0.h5']
['step 1 - loss 2.555248737335205']
conv1 : 
[ 0.3550244  -0.22819042 -0.59257895  0.018972    0.06619009  0.35148212
  0.0565969   0.49648634  0.5115059   0.13359042 -0.0953503  -0.21491151
 -0.70027059  0.57943124  0.18562542 -0.28042778]
Saved weigths :  yolo-tiny-step1.h5
['step 2 - loss 2.632427215576172']
conv1 : 
[ 0.35498244 -0.22823356 -0.59256691  0.01902446  0.06609867  0.35149047
  0.05664805  0.49655351  0.51160401  0.13351972 -0.0953752  -0.21491593
 -0.70022213  0.57952332  0.1855337  -0.28048456]
['step 3 - loss 2.460310935974121']
conv1 : 
[ 0.35495365 -0.22826865 -0.59253889  0.01902069  0.06601952  0.351508

KeyboardInterrupt: 

In [10]:
sess.close()

In [12]:
def printname(name):
    print(name)

f = h5py.File(weights_path)
f.visit(printname)
f.close()

convolution2d_1
convolution2d_1/convolution2d_1_W:0
convolution2d_1/convolution2d_1_b:0
convolution2d_2
convolution2d_2/convolution2d_2_W:0
convolution2d_2/convolution2d_2_b:0
convolution2d_3
convolution2d_3/convolution2d_3_W:0
convolution2d_3/convolution2d_3_b:0
convolution2d_4
convolution2d_4/convolution2d_4_W:0
convolution2d_4/convolution2d_4_b:0
convolution2d_5
convolution2d_5/convolution2d_5_W:0
convolution2d_5/convolution2d_5_b:0
convolution2d_6
convolution2d_6/convolution2d_6_W:0
convolution2d_6/convolution2d_6_b:0
convolution2d_7
convolution2d_7/convolution2d_7_W:0
convolution2d_7/convolution2d_7_b:0
convolution2d_8
convolution2d_8/convolution2d_8_W:0
convolution2d_8/convolution2d_8_b:0
convolution2d_9
convolution2d_9/convolution2d_9_W:0
convolution2d_9/convolution2d_9_b:0
dense_1
dense_1/dense_1_W:0
dense_1/dense_1_b:0
dense_2
dense_2/dense_2_W:0
dense_2/dense_2_b:0
dense_3
dense_3/dense_3_W:0
dense_3/dense_3_b:0
flatten_1
leakyrelu_10
maxpooling2d_1
maxpooling2d_2
maxpooling2